# 1- Let's start by importing the needed modules

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 2- Connect  to foursquare 

In [3]:
CLIENT_ID = 'VQ42RGRQWCGTXCTZBIH4U335JJM3EJPGMO2QJGJCZXJJBPGH' # your Foursquare ID
CLIENT_SECRET = 'KRVCTRRCFNTKCTZ2DCDV4LIACISZRUDW4ARAAKMNGC5SEEAF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VQ42RGRQWCGTXCTZBIH4U335JJM3EJPGMO2QJGJCZXJJBPGH
CLIENT_SECRET:KRVCTRRCFNTKCTZ2DCDV4LIACISZRUDW4ARAAKMNGC5SEEAF


# 3- Get the coordinates of Paris

In [4]:
city = 'Paris'
geolocator = Nominatim()    
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

C:\Users\Mrabah\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


48.8566101 2.3514992


# 4- Get some very important data from opendata.paris.fr
## This csv file contains all types of commercial activities like restaurants, sport centers etc

In [13]:
paris_data = pd.read_csv("https://opendata.paris.fr/explore/dataset/commercesparis/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true",sep=';',index_col = 0)
paris_data.head()

C:\Users\Mrabah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ARRONDISSEMENT,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV
ORDRE,,,,,,,,,,,,,,,,,,,,
2379.0,75001.0,3.0,751010301.0,7.510103e+10,3.0,NaN,RUE,PYRAMIDES,3 RUE PYRAMIDES,7902,651020.0953,6.862905e+06,"48.864299, 2.332401",3.0,Sur rue,CH106,Restaurant européen,1.0,NaN,NaN
2381.0,75001.0,3.0,751010301.0,7.510103e+10,5.0,NaN,RUE,PYRAMIDES,5 RUE PYRAMIDES,7902,651025.2825,6.862915e+06,"48.864414, 2.332488",1.0,Sur rue,CH403,Bar ou Café sans tabac,1.0,NaN,NaN
2388.0,75001.0,3.0,751010301.0,7.510103e+10,194.0,NaN,RUE,RIVOLI,194 RUE RIVOLI,8249,650976.9246,6.862871e+06,"48.863995, 2.331651",4.0,En angle de rues,SB204,Bureau de change,1.0,NaN,NaN
2389.0,75001.0,3.0,751010301.0,7.510103e+10,189.0,NaN,RUE,SAINT HONORE,189 RUE SAINT HONORE,8635,651027.8751,6.862936e+06,"48.864601, 2.33244",1.0,En angle de rues,CB105,Prêt-à-porter Homme,1.0,NaN,NaN
2396.0,75001.0,3.0,751010301.0,7.510103e+10,195.0,NaN,RUE,SAINT HONORE,195 RUE SAINT HONORE,8635,651000.9718,6.862956e+06,"48.864777, 2.332058",1.0,Sur rue,CC202,Parfumerie - Produits de beauté,1.0,NaN,NaN


# 5- Let's drop all rows that don't have a relation with a restaurant or a sport center

In [15]:
paris_data['Rest'] = paris_data['LIBELLE ACTIVITE'].str.startswith('Restaura') 
paris_data['sports'] = paris_data['LIBELLE ACTIVITE'].str.startswith('Salle de sport')
paris_data.head()

,ARRONDISSEMENT,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV,Rest,sports
ORDRE,,,,,,,,,,,,,,,,,,,,,,
2379.0,75001.0,3.0,751010301.0,7.510103e+10,3.0,NaN,RUE,PYRAMIDES,3 RUE PYRAMIDES,7902,651020.0953,6.862905e+06,"48.864299, 2.332401",3.0,Sur rue,CH106,Restaurant européen,1.0,NaN,NaN,True,False
2381.0,75001.0,3.0,751010301.0,7.510103e+10,5.0,NaN,RUE,PYRAMIDES,5 RUE PYRAMIDES,7902,651025.2825,6.862915e+06,"48.864414, 2.332488",1.0,Sur rue,CH403,Bar ou Café sans tabac,1.0,NaN,NaN,False,False
2388.0,75001.0,3.0,751010301.0,7.510103e+10,194.0,NaN,RUE,RIVOLI,194 RUE RIVOLI,8249,650976.9246,6.862871e+06,"48.863995, 2.331651",4.0,En angle de rues,SB204,Bureau de change,1.0,NaN,NaN,False,False
2389.0,75001.0,3.0,751010301.0,7.510103e+10,189.0,NaN,RUE,SAINT HONORE,189 RUE SAINT HONORE,8635,651027.8751,6.862936e+06,"48.864601, 2.33244",1.0,En angle de rues,CB105,Prêt-à-porter Homme,1.0,NaN,NaN,False,False
2396.0,75001.0,3.0,751010301.0,7.510103e+10,195.0,NaN,RUE,SAINT HONORE,195 RUE SAINT HONORE,8635,651000.9718,6.862956e+06,"48.864777, 2.332058",1.0,Sur rue,CC202,Parfumerie - Produits de beauté,1.0,NaN,NaN,False,False


## 5.1 only rows with a relation to a restaurant

In [16]:
paris_rest = paris_data.drop(paris_data.loc[paris_data['Rest']!=True].index)
paris_rest.head()

,ARRONDISSEMENT,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV,Rest,sports
ORDRE,,,,,,,,,,,,,,,,,,,,,,
2379.0,75001.0,3.0,751010301.0,7.510103e+10,3.0,NaN,RUE,PYRAMIDES,3 RUE PYRAMIDES,7902,651020.0953,6.862905e+06,"48.864299, 2.332401",3.0,Sur rue,CH106,Restaurant européen,1.0,NaN,NaN,True,False
2477.0,75001.0,4.0,751010402.0,7.510104e+10,4.0,NaN,RUE,LA SOURDIERE,4 RUE LA SOURDIERE,5368,650979.0443,6.863011e+06,"48.86526, 2.331778",1.0,Sur rue,CH101,Restaurant traditionnel français,1.0,NaN,NaN,True,False
2507.0,75001.0,4.0,751010402.0,7.510104e+10,33.0,NaN,RUE,SAINT ROCH,33 RUE SAINT ROCH,8740,651035.7979,6.863086e+06,"48.865932, 2.332541",1.0,Sur rue,CH101,Restaurant traditionnel français,1.0,NaN,NaN,True,False
2508.0,75001.0,4.0,751010402.0,7.510104e+10,39.0,NaN,RUE,SAINT ROCH,39 RUE SAINT ROCH,8740,651053.2790,6.863125e+06,"48.866285, 2.332785",1.0,Sur rue,CH108,"Restaurant indien, pakistanais et Moyen Orient",1.0,NaN,NaN,True,False
2527.0,75001.0,4.0,751010402.0,7.510104e+10,24.0,NaN,PL,MARCHE SAINT HONORE,24 PL MARCHE SAINT HONORE,6033,650997.2031,6.863143e+06,"48.866444, 2.332008",2.0,Sur rue,CH101,Restaurant traditionnel français,1.0,NaN,NaN,True,False


## 5.2 Group them by arrondissement

In [18]:
# group all restaurants by arrondissement (neighborhood)
nb_rest = paris_rest.groupby('ARRONDISSEMENT').count()
nb_rest.head()

,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV,Rest,sports
ARRONDISSEMENT,,,,,,,,,,,,,,,,,,,,,
75001.0,358,358,358,358,7,358,358,358,358,358,358,358,358,358,358,358,358,32,25,358,358
75002.0,361,361,361,361,8,361,361,361,361,361,361,361,361,361,361,361,361,33,32,361,361
75003.0,201,201,201,201,0,201,201,201,201,201,201,201,201,201,201,201,201,5,4,201,201
75004.0,221,221,221,221,5,221,221,221,221,221,221,221,221,221,221,221,221,3,0,221,221
75005.0,495,495,495,495,8,495,495,495,495,495,495,495,495,495,495,495,495,6,0,495,495


In [19]:
# number of restaurants in each arrondissement, from 1 to 20
nb_rest = list(nb_rest['QUARTIER'])
nb_rest

[358,
 361,
 201,
 221,
 495,
 359,
 216,
 591,
 669,
 512,
 608,
 403,
 443,
 386,
 609,
 343,
 595,
 585,
 299,
 284]

## 5.3 only rows with a relation to sport centers

In [17]:
paris_sports = paris_data.drop(paris_data.loc[paris_data['sports']!=True].index)
paris_sports.head()

,ARRONDISSEMENT,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV,Rest,sports
ORDRE,,,,,,,,,,,,,,,,,,,,,,
39472.0,75011.0,43.0,751114310.0,7.511143e+10,5.0,NaN,CITE,ROQUETTE,5 CITE ROQUETTE,8357,654085.8149,6.861829e+06,"48.854902, 2.3742",2.0,Sur rue,SA406,Salle de sport spécialisée,1.0,NaN,NaN,False,True
52058.0,75014.0,56.0,751145623.0,7.511456e+10,44.0,NaN,RUE,RAYMOND LOSSERAND,44 RUE RAYMOND LOSSERAND,8061,650072.9072,6.859644e+06,"48.834891, 2.319809",1.0,Sur rue,SA406,Salle de sport spécialisée,1.0,NaN,NaN,False,True
23265.0,75008.0,31.0,751083106.0,7.510831e+10,49.0,NaN,RUE,MATHURINS,49 RUE MATHURINS,6176,650347.4163,6.863913e+06,"48.87333, 2.323054",1.0,Sur rue,SA406,Salle de sport spécialisée,1.0,NaN,NaN,False,True
13014.0,75005.0,19.0,751051906.0,7.510519e+10,179.0,NaN,RUE,SAINT JACQUES,179 RUE SAINT JACQUES,8648,651762.1848,6.860808e+06,"48.845498, 2.3427",1.0,En cours d'immeuble,SA406,Salle de sport spécialisée,1.0,NaN,NaN,False,True
67520.0,75017.0,67.0,751176710.0,7.511767e+10,106.0,NaN,RUE,CARDINET,106 RUE CARDINET,1550,649514.0266,6.865272e+06,"48.885469, 2.31153",3.0,Sur rue,SA406,Salle de sport spécialisée,1.0,NaN,NaN,False,True


## 5.4 Group them by arrondissement

In [20]:
nb_sports = paris_sports.groupby('ARRONDISSEMENT').count()
nb_sports.head()

,QUARTIER,IRIS,ILOT,NUMERO,LET,TYPE VOIE,LIBELLE VOIE,ADRESSE COMPLETE,CFA,X,Y,XY,SEQUENCE,SITUATION,CODE ACTIVITE,LIBELLE ACTIVITE,SURFACE,CC ID,CC NIV,Rest,sports
ARRONDISSEMENT,,,,,,,,,,,,,,,,,,,,,
75001.0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1
75002.0,3,3,3,3,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,3,3
75003.0,6,6,6,6,1,6,6,6,6,6,6,6,6,6,6,6,6,1,0,6,6
75004.0,4,4,4,4,0,4,4,4,4,4,4,4,4,4,4,4,4,0,0,4,4
75005.0,6,6,6,6,2,6,6,6,6,6,6,6,6,6,6,6,6,1,0,6,6


In [21]:
nb_sports = list(nb_sports['QUARTIER'])

# 6- Let's create our clean dataframe that will contain our final clean data

In [22]:
new_cols = ['Neighborhood','Nb restaurants','Nb sport centers','Nb parks','Nb stations','price per squared m']
paris = pd.DataFrame(columns=new_cols)
paris

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m


## 6.1 Fill the neighborhood column from 1 to 20

In [23]:
paris['Neighborhood'] = [*range(1,21)]


## 6.1 Fill the Nb restaurant column with the obtained list in step 5.2

In [24]:
paris['Nb restaurants'] = nb_rest

In [25]:
paris.head()

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m
0,1,358,NaN,NaN,NaN,NaN
1,2,361,NaN,NaN,NaN,NaN
2,3,201,NaN,NaN,NaN,NaN
3,4,221,NaN,NaN,NaN,NaN
4,5,495,NaN,NaN,NaN,NaN


## 6.2 Fill the Nb sport centers column with the obtained list in step 5.4

In [26]:
paris['Nb sport centers'] = nb_sports
paris.head()

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m
0,1,358,1,NaN,NaN,NaN
1,2,361,3,NaN,NaN,NaN
2,3,201,6,NaN,NaN,NaN
3,4,221,4,NaN,NaN,NaN
4,5,495,6,NaN,NaN,NaN


## 6.3 Get data from the same parisdata website, we will extract the coordinates of each arrondissement from it

In [36]:
coord = pd.read_csv("https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true", sep=';',index_col = 0)
coord.head()

,C_AR,C_ARINSEE,L_AR,L_AROFF,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
N_SQ_AR,,,,,,,,,,,
750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225, 2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764
750000003,3,75103,3ème Ardt,Temple,750001537,1.170883e+06,4519.263648,"48.86287238, 2.3600009859","{""type"": ""Polygon"", ""coordinates"": [[[2.363828...",3,4519.071982
750000012,12,75112,12ème Ardt,Reuilly,750001537,1.631478e+07,24089.666298,"48.8349743815, 2.42132490078","{""type"": ""Polygon"", ""coordinates"": [[[2.413879...",12,24088.038922
750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018, 2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862
750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263, 2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779


## 6.4 Fill the latitude and longitude columns

In [45]:
latitude = []
longitude = []
coord = coord[['C_ARINSEE','Geometry X Y']]
coord.sort_values(by=['C_ARINSEE'],inplace = True)
for i in range(coord.shape[0]):
    latitude.append(coord.iloc[i,1].split(',')[0])
    longitude.append(coord.iloc[i,1].split(',')[1])
paris['latitude'] = latitude
paris['longitude'] = longitude
paris.head()

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,358,1,NaN,NaN,NaN,48.8625627018,2.33644336205
1,2,361,3,NaN,NaN,NaN,48.8682792225,2.34280254689
2,3,201,6,NaN,NaN,NaN,48.86287238,2.3600009859
3,4,221,4,NaN,NaN,NaN,48.8543414263,2.35762962032
4,5,495,6,NaN,NaN,NaN,48.8444431505,2.35071460958


## 6.5 Here, I downloaded the original csv file to my local storage, dropped the non needed columns and imported the final version of the csv file

In [86]:
paris_parks = pd.read_csv("C:\\Users\\Mrabah\\Desktop\\SQL IBM DATA SCIENCE\\Capstone project Paris\\paris_parks.csv", sep = ';', index_col = 0)
paris_parks.head()

,NOM JARDIN,ADRESSE JARDIN,ARRONDISSEMENT
0,Square Auguste Renoir,Rue des Mariniers,14
1,Le Jardin aux Habitants,Palais de Tokyo / Rue de la Manutention,16
2,Jardins du Ruisseau,PC / à côté du 110 rue du Ruisseau,18
3,Un P'tit Bol d'Air,11 quai de l'Oise,19
4,Potager des Oiseaux,Rue des Oiseaux,3


## 6.6 Group the parks by arrondissement to obtain the number of parks in each one of them

In [87]:
parks = paris_parks.groupby('ARRONDISSEMENT').count()
parks = parks.reset_index()
parks

,ARRONDISSEMENT,NOM JARDIN,ADRESSE JARDIN
0,1,1,1
1,2,1,1
2,3,2,2
3,4,1,1
4,8,1,1
5,10,5,5
6,11,6,6
7,12,6,6
8,13,8,8
9,14,8,8


## 6.7 The 5th, 6th,7th and 9th arrondissement don't contain any parks, so I appended them to the dataframe with a value of 0

In [88]:
missed = [5,6,7,9]
for k in missed:
    parks.loc[-1] = [k, 0,0]  # adding a row
    parks.index = parks.index + 1  # shifting index
    parks = parks.sort_index()  # sorting by index
parks

,ARRONDISSEMENT,NOM JARDIN,ADRESSE JARDIN
0,9,0,0
1,7,0,0
2,6,0,0
3,5,0,0
4,1,1,1
5,2,1,1
6,3,2,2
7,4,1,1
8,8,1,1
9,10,5,5


In [89]:
parks.sort_values(by=['ARRONDISSEMENT'], inplace = True)
parks = parks.reset_index( drop = True)
parks

,ARRONDISSEMENT,NOM JARDIN,ADRESSE JARDIN
0,1,1,1
1,2,1,1
2,3,2,2
3,4,1,1
4,5,0,0
5,6,0,0
6,7,0,0
7,8,1,1
8,9,0,0
9,10,5,5


## 6.8 Fill the Nb parks column

In [90]:
paris['Nb parks'] = list(parks['NOM JARDIN'])
paris

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,358,1,1,NaN,NaN,48.8625627018,2.33644336205
1,2,361,3,1,NaN,NaN,48.8682792225,2.34280254689
2,3,201,6,2,NaN,NaN,48.86287238,2.3600009859
3,4,221,4,1,NaN,NaN,48.8543414263,2.35762962032
4,5,495,6,0,NaN,NaN,48.8444431505,2.35071460958
5,6,359,5,0,NaN,NaN,48.8491303586,2.33289799905
6,7,216,5,0,NaN,NaN,48.8561744288,2.31218769148
7,8,591,7,1,NaN,NaN,48.8727208374,2.3125540224
8,9,669,8,0,NaN,NaN,48.8771635173,2.33745754348
9,10,512,2,5,NaN,NaN,48.8761300365,2.36072848785


## 6.9 I did the same thing as in step 6.5 with the csv file containing the paris metro and train stations

In [94]:
stations = pd.read_csv("C:\\Users\\Mrabah\\Desktop\\SQL IBM DATA SCIENCE\\Capstone project Paris\\paris_stations.csv", sep = ';', index_col = 0)
stations.head()

,stop_name,Arrondissement
0,Alésia,14
1,Concorde,8
2,Convention,15
3,Denfert-Rochereau,14
4,Gare de Lyon,12


In [95]:
stations = stations.groupby('Arrondissement').count()
stations = stations.reset_index()
stations

,Arrondissement,stop_name
0,1,219
1,2,106
2,3,93
3,4,219
4,5,250
5,6,285
6,7,278
7,8,471
8,9,280
9,10,326


## 6.10 Fill the Nb stations column

In [96]:
paris['Nb stations'] = list(stations['stop_name'])
paris

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,358,1,1,219,NaN,48.8625627018,2.33644336205
1,2,361,3,1,106,NaN,48.8682792225,2.34280254689
2,3,201,6,2,93,NaN,48.86287238,2.3600009859
3,4,221,4,1,219,NaN,48.8543414263,2.35762962032
4,5,495,6,0,250,NaN,48.8444431505,2.35071460958
5,6,359,5,0,285,NaN,48.8491303586,2.33289799905
6,7,216,5,0,278,NaN,48.8561744288,2.31218769148
7,8,591,7,1,471,NaN,48.8727208374,2.3125540224
8,9,669,8,0,280,NaN,48.8771635173,2.33745754348
9,10,512,2,5,326,NaN,48.8761300365,2.36072848785


## 6.11 I did the same thing as in step 6.5 and 6.9 with the csv file containing the mean price of a squared meter in each arrondissement

In [97]:
price = pd.read_csv("C:\\Users\\Mrabah\\Desktop\\SQL IBM DATA SCIENCE\\Capstone project Paris\\paris_price.csv", sep = ';', index_col = 0)
price.head()

,Arrondissement,Price
0,1,9950
1,2,8970
2,3,9900
3,4,10310
4,5,10380


## 6.12 Fill the price per squared m column

In [99]:
paris['price per squared m'] = list(price['Price'])
paris

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,358,1,1,219,9950,48.8625627018,2.33644336205
1,2,361,3,1,106,8970,48.8682792225,2.34280254689
2,3,201,6,2,93,9900,48.86287238,2.3600009859
3,4,221,4,1,219,10310,48.8543414263,2.35762962032
4,5,495,6,0,250,10380,48.8444431505,2.35071460958
5,6,359,5,0,285,11710,48.8491303586,2.33289799905
6,7,216,5,0,278,10710,48.8561744288,2.31218769148
7,8,591,7,1,471,9920,48.8727208374,2.3125540224
8,9,669,8,0,280,8860,48.8771635173,2.33745754348
9,10,512,2,5,326,7500,48.8761300365,2.36072848785


In [100]:
# Here I saved this pandas dataframe to my local storage to avoid losing it 
paris.to_csv("C:\\Users\\Mrabah\\Desktop\\SQL IBM DATA SCIENCE\\Capstone project Paris\\final_data.csv", sep = ';')

In [164]:
## Read the dataframe
data = pd.read_csv("C:\\Users\\Mrabah\\Desktop\\SQL IBM DATA SCIENCE\\Capstone project Paris\\final_data.csv",sep=';',index_col = 0)
data

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,358,1,1,219,9950,48.862563,2.336443
1,2,361,3,1,106,8970,48.868279,2.342803
2,3,201,6,2,93,9900,48.862872,2.360001
3,4,221,4,1,219,10310,48.854341,2.357630
4,5,495,6,0,250,10380,48.844443,2.350715
5,6,359,5,0,285,11710,48.849130,2.332898
6,7,216,5,0,278,10710,48.856174,2.312188
7,8,591,7,1,471,9920,48.872721,2.312554
8,9,669,8,0,280,8860,48.877164,2.337458
9,10,512,2,5,326,7500,48.876130,2.360728


## 7- let's normalize features
## n = (X - Xmin) / (Xmax - Xmin)
### for the price per squared meter, we need to calculate 1 - the normalized value
### The lower the price is, the more interesting it is for the company

In [165]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns = ['Nb restaurants','Nb sport centers','Nb parks','Nb stations','price per squared m']
data[columns] = scaler.fit_transform(data[columns])

data
        

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,0.335470,0.000000,0.055556,0.306569,0.654224,48.862563,2.336443
1,2,0.341880,0.095238,0.055556,0.031630,0.461690,48.868279,2.342803
2,3,0.000000,0.238095,0.111111,0.000000,0.644401,48.862872,2.360001
3,4,0.042735,0.142857,0.055556,0.306569,0.724951,48.854341,2.357630
4,5,0.628205,0.238095,0.000000,0.381995,0.738703,48.844443,2.350715
5,6,0.337607,0.190476,0.000000,0.467153,1.000000,48.849130,2.332898
6,7,0.032051,0.190476,0.000000,0.450122,0.803536,48.856174,2.312188
7,8,0.833333,0.285714,0.055556,0.919708,0.648330,48.872721,2.312554
8,9,1.000000,0.333333,0.000000,0.454988,0.440079,48.877164,2.337458
9,10,0.664530,0.047619,0.277778,0.566910,0.172888,48.876130,2.360728


In [166]:
data['price per squared m'] = 1 - np.array(data['price per squared m'])
data

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude
0,1,0.335470,0.000000,0.055556,0.306569,3.457760e-01,48.862563,2.336443
1,2,0.341880,0.095238,0.055556,0.031630,5.383104e-01,48.868279,2.342803
2,3,0.000000,0.238095,0.111111,0.000000,3.555992e-01,48.862872,2.360001
3,4,0.042735,0.142857,0.055556,0.306569,2.750491e-01,48.854341,2.357630
4,5,0.628205,0.238095,0.000000,0.381995,2.612967e-01,48.844443,2.350715
5,6,0.337607,0.190476,0.000000,0.467153,-2.220446e-16,48.849130,2.332898
6,7,0.032051,0.190476,0.000000,0.450122,1.964637e-01,48.856174,2.312188
7,8,0.833333,0.285714,0.055556,0.919708,3.516699e-01,48.872721,2.312554
8,9,1.000000,0.333333,0.000000,0.454988,5.599214e-01,48.877164,2.337458
9,10,0.664530,0.047619,0.277778,0.566910,8.271120e-01,48.876130,2.360728


## 8- let's define the weight of each feature, definded by the company (priority of each feature)

| Feature        | weight |
|--------------------|---------|
| price per squared m               | 4   |
| Nb stations      | 2    | 
| Nb restaurants               | 2    |
| Nb sport centers | 1   |
| Nb parks |1|

In [227]:
# Take the needed columns to calculate the settlement index
data_to_use = data[columns]
data_to_use.shape

(20, 5)

## 8.1 We calculate the settlement index, it is a valur that varies between 1 and 10, the closer the value is to 10, the better is the corresponding arrondisement to the company

## If the value is close to 1, the company should absolutely avoid the corresponding neighborhoods

In [187]:
indexes = [*range(data_to_use.shape[1])]
weights = [2, 1, 1, 2, 4]
choice_idx = []
for i in range(data_to_use.shape[0]):
    res = 0
    for w,j in zip(weights, indexes):
        res+= data_to_use.iloc[i,j] * w
    choice_idx.append("%.2f" % res)

In [192]:
idx = [float(i) for i in choice_idx]
idx

[2.72,
 3.05,
 1.77,
 2.0,
 3.3,
 1.8,
 1.94,
 5.25,
 5.48,
 6.1,
 6.24,
 6.8,
 6.4,
 5.75,
 7.43,
 5.04,
 6.91,
 7.8,
 6.41,
 6.69]

In [196]:
data['settlement index'] = idx
data_to_use['settlement index'] = idx


C:\Users\Mrabah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## 8.2 We add the obtained indexes as a new column to our dataframe

In [195]:
paris['settlement index'] = idx
paris

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude,settlement index
0,1,358,1,1,219,9950,48.8625627018,2.33644336205,2.72
1,2,361,3,1,106,8970,48.8682792225,2.34280254689,3.05
2,3,201,6,2,93,9900,48.86287238,2.3600009859,1.77
3,4,221,4,1,219,10310,48.8543414263,2.35762962032,2.00
4,5,495,6,0,250,10380,48.8444431505,2.35071460958,3.30
5,6,359,5,0,285,11710,48.8491303586,2.33289799905,1.80
6,7,216,5,0,278,10710,48.8561744288,2.31218769148,1.94
7,8,591,7,1,471,9920,48.8727208374,2.3125540224,5.25
8,9,669,8,0,280,8860,48.8771635173,2.33745754348,5.48
9,10,512,2,5,326,7500,48.8761300365,2.36072848785,6.10


In [199]:
## sort the dataframe by the settlement index
paris_sorted = paris.sort_values(by=['settlement index'])
paris_sorted = paris_sorted.reset_index( drop = True)
paris_sorted

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude,settlement index
0,3,201,6,2,93,9900,48.86287238,2.3600009859,1.77
1,6,359,5,0,285,11710,48.8491303586,2.33289799905,1.80
2,7,216,5,0,278,10710,48.8561744288,2.31218769148,1.94
3,4,221,4,1,219,10310,48.8543414263,2.35762962032,2.00
4,1,358,1,1,219,9950,48.8625627018,2.33644336205,2.72
5,2,361,3,1,106,8970,48.8682792225,2.34280254689,3.05
6,5,495,6,0,250,10380,48.8444431505,2.35071460958,3.30
7,16,343,13,1,455,9110,48.8603921054,2.26197078836,5.04
8,8,591,7,1,471,9920,48.8727208374,2.3125540224,5.25
9,9,669,8,0,280,8860,48.8771635173,2.33745754348,5.48


## 9- We can classify our ressults into 3 groups based on the settlement index value:
###    if settlement index > 6 : The company is advised to establish its new office in the correspondant neighborhoods
###    if settlement index is between 4 and 6 : It can be a solution
###    if settlement index < 4 : The company should absolutely avoid these choices

### We found that : arrondissements : 1 to 7 should be avoided (< 4)
###                            arrondissements : 16, 8, 9, 14 can be a solution ( 4< index < 6)
###                            arrondissements : 10, 11, 13, 19, 20, 12, 17, 15, 18 are a good solution (index > 6)

In [204]:
data

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude,settlement index
0,1,0.335470,0.000000,0.055556,0.306569,3.457760e-01,48.862563,2.336443,2.72
1,2,0.341880,0.095238,0.055556,0.031630,5.383104e-01,48.868279,2.342803,3.05
2,3,0.000000,0.238095,0.111111,0.000000,3.555992e-01,48.862872,2.360001,1.77
3,4,0.042735,0.142857,0.055556,0.306569,2.750491e-01,48.854341,2.357630,2.00
4,5,0.628205,0.238095,0.000000,0.381995,2.612967e-01,48.844443,2.350715,3.30
5,6,0.337607,0.190476,0.000000,0.467153,-2.220446e-16,48.849130,2.332898,1.80
6,7,0.032051,0.190476,0.000000,0.450122,1.964637e-01,48.856174,2.312188,1.94
7,8,0.833333,0.285714,0.055556,0.919708,3.516699e-01,48.872721,2.312554,5.25
8,9,1.000000,0.333333,0.000000,0.454988,5.599214e-01,48.877164,2.337458,5.48
9,10,0.664530,0.047619,0.277778,0.566910,8.271120e-01,48.876130,2.360728,6.10


# 10- Let's compare our results to the results that we are going to obtain by clustering

In [212]:
final_columns= ['Neighborhood'] + columns 
final_data = data[final_columns]
final_data

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m
0,1,0.335470,0.000000,0.055556,0.306569,3.457760e-01
1,2,0.341880,0.095238,0.055556,0.031630,5.383104e-01
2,3,0.000000,0.238095,0.111111,0.000000,3.555992e-01
3,4,0.042735,0.142857,0.055556,0.306569,2.750491e-01
4,5,0.628205,0.238095,0.000000,0.381995,2.612967e-01
5,6,0.337607,0.190476,0.000000,0.467153,-2.220446e-16
6,7,0.032051,0.190476,0.000000,0.450122,1.964637e-01
7,8,0.833333,0.285714,0.055556,0.919708,3.516699e-01
8,9,1.000000,0.333333,0.000000,0.454988,5.599214e-01
9,10,0.664530,0.047619,0.277778,0.566910,8.271120e-01


## 10.1 each column was multiplied by the corresponding weight, we need these updated values for clustering

In [213]:
final_data['Nb restaurants'] = [float(i) for i in list(np.array(final_data['Nb restaurants']) * 2)]
final_data['Nb sport centers'] = [float(i) for i in list(np.array(final_data['Nb sport centers']) * 1)]
final_data['Nb parks'] = [float(i) for i in list(np.array(final_data['Nb parks']) * 1)]
final_data['Nb stations'] = [float(i) for i in list(np.array(final_data['Nb stations']) * 2)]
final_data['price per squared m'] = [float(i) for i in list(np.array(final_data['price per squared m']) * 4)]
final_data


C:\Users\Mrabah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Mrabah\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Mrabah\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m
0,1,0.670940,0.000000,0.055556,0.613139,1.383104e+00
1,2,0.683761,0.095238,0.055556,0.063260,2.153242e+00
2,3,0.000000,0.238095,0.111111,0.000000,1.422397e+00
3,4,0.085470,0.142857,0.055556,0.613139,1.100196e+00
4,5,1.256410,0.238095,0.000000,0.763990,1.045187e+00
5,6,0.675214,0.190476,0.000000,0.934307,-8.881784e-16
6,7,0.064103,0.190476,0.000000,0.900243,7.858546e-01
7,8,1.666667,0.285714,0.055556,1.839416,1.406680e+00
8,9,2.000000,0.333333,0.000000,0.909976,2.239686e+00
9,10,1.329060,0.047619,0.277778,1.133820,3.308448e+00


## 10.2 We launch clustering analysis for (n clusters = 3)

In [214]:
clustering_data = final_data.drop('Neighborhood',1)

kclusters = 3

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustering_data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 2, 1])

In [215]:
# obtained labels
cluster_labels = kmeans.labels_
cluster_labels

array([0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1])

## 10.3 We add the cluster label column to the dataframe

In [217]:
data['cluster label'] = cluster_labels
data

,Neighborhood,Nb restaurants,Nb sport centers,Nb parks,Nb stations,price per squared m,latitude,longitude,settlement index,cluster label
0,1,0.335470,0.000000,0.055556,0.306569,3.457760e-01,48.862563,2.336443,2.72,0
1,2,0.341880,0.095238,0.055556,0.031630,5.383104e-01,48.868279,2.342803,3.05,0
2,3,0.000000,0.238095,0.111111,0.000000,3.555992e-01,48.862872,2.360001,1.77,0
3,4,0.042735,0.142857,0.055556,0.306569,2.750491e-01,48.854341,2.357630,2.00,0
4,5,0.628205,0.238095,0.000000,0.381995,2.612967e-01,48.844443,2.350715,3.30,0
5,6,0.337607,0.190476,0.000000,0.467153,-2.220446e-16,48.849130,2.332898,1.80,0
6,7,0.032051,0.190476,0.000000,0.450122,1.964637e-01,48.856174,2.312188,1.94,0
7,8,0.833333,0.285714,0.055556,0.919708,3.516699e-01,48.872721,2.312554,5.25,2
8,9,1.000000,0.333333,0.000000,0.454988,5.599214e-01,48.877164,2.337458,5.48,2
9,10,0.664530,0.047619,0.277778,0.566910,8.271120e-01,48.876130,2.360728,6.10,1


## Conclusion & results : We can clearly see in the table above that neighborhoods belonging to the label 1 and label 2 are corresponding to the first test where we calculated the settlement index, all neighborhoods that have an index > 4 belong to the 1st or the 2nd label

## This clustering analysis provides us with more certainty, because it also showed us that we need to avoid neighborhoods that have an index < 4 (label 0 after clustering ), we obtained the same results after clustering

## 10.4 Generate the map of arrondissements

In [226]:
# create map
latitude = 48.8566101
longitude = 2.3514992

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon,  cluster in zip(data['latitude'], data['longitude'], data['cluster label']):
    label = folium.Popup(cluster, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## As we can see from the map, the red neighborhoods are at the center of paris, and it is known that they are the most expensive ones, that's why the clustering analysis predicted them as the ones that the company should avoid for its office, and the other neighborhoods are around the center, and they are less expensive with a good accessibility to restaurants, parks, stations and sport centers.

## As a suggestion, the company should establish the new office at the 18th arrondissement as it has the highest settlement index